In [1]:
from stravalib.client import Client
import pandas as pd
from geopy.distance import vincenty
from retrying import retry
import time
import numpy as np
from tqdm import tqdm

client = Client(access_token='76824abf6abf903eb3d8b0bde83625135c0be0ec')
athlete = client.get_athlete()

ImportError: No module named stravalib.client

In [2]:
@retry(wait_exponential_multiplier=1000, wait_exponential_max=10000, stop_max_delay=30000)
def retry_get_leaderboard(client,segment):
    total = client.get_segment_leaderboard(segment)
    friends = client.get_segment_leaderboard(segment,following=True)
    c = (total,friends)
    return c

@retry(wait_exponential_multiplier=1000, wait_exponential_max=10000, stop_max_delay=30000)
def retry_get_segment(client,segment):
    return client.get_segment(segment)

In [3]:
df = pd.read_csv('segoutput.csv',index_col=False)

#home = (45.487803,-73.576913)
home = (45.502192, -73.577163)
df['coordinates'] = zip(df['latitude'], df['longitude'])

biglist = df.T.to_dict().values()
for row in biglist:
    row['distance_from_home'] = vincenty(row['coordinates'], home).km

new_df = pd.DataFrame(biglist)
new_df = new_df[new_df['distance_from_home'] <= 2].sort_values(['distance_from_home'], ascending=[True])

smalllist = new_df.T.to_dict().values()

for row in tqdm(smalllist):
    
    time.sleep(2)
        
    row['segment_distance'] = float(retry_get_segment(client,row["segment_id"]).distance)
    
    both_leaderboards = retry_get_leaderboard(client,row["segment_id"]) 
    row['total_top_time'] = both_leaderboards[0][0].elapsed_time
    
    times = both_leaderboards[0][0].elapsed_time
    for x in both_leaderboards[0][1:]:
        times += x.elapsed_time
    times = times / len(both_leaderboards[0])
    row['average_total_top_time'] = times
    
    
    if not both_leaderboards[1]:
        row['friend_top_time'] = np.nan        
    else:
        row['friend_top_time'] = both_leaderboards[1][0].elapsed_time
        
final_df = pd.DataFrame(smalllist)
final_df['friend_versus_top1'] = (final_df['friend_top_time'] - final_df['total_top_time'])/final_df['total_top_time']*100
final_df['friend_versus_top10'] = (final_df['friend_top_time'] - final_df['average_total_top_time'])/final_df['average_total_top_time']*100


In [4]:
final_df['distance_from_home_Ranked'] = final_df['distance_from_home'].rank(ascending=1)
final_df['segment_distance_Ranked'] = final_df['segment_distance'].rank(ascending=1)
final_df['friend_versus_top10_Ranked'] = final_df['friend_versus_top10'].rank(ascending=0)
final_df['target_score'] = 0.2*final_df['distance_from_home_Ranked'] + 0.1*final_df['segment_distance_Ranked'] + 0.7*final_df['friend_versus_top10_Ranked']
print final_df.sort_values(['target_score'], ascending=[True])

   average_total_top_time   color                      coordinates  \
4         00:00:58.900000  5CB1E4           (45.510657, -73.59342)   
1         00:00:36.900000  5CB1E4          (45.513068, -73.589766)   
7         00:00:29.700000  FFEE33  (45.5170843843, -73.5649731383)   
3         00:04:42.700000  FFEE33          (45.488943, -73.594291)   
0                00:03:51  5CB1E4  (45.5146049336, -73.5905919224)   
6         00:00:50.200000  FFEE33  (45.5123269149, -73.5792665133)   
8         00:01:01.300000  FFFFFF  (45.5160342471, -73.5625345916)   
5         00:00:24.500000  5CB1E4          (45.499057, -73.601005)   
2         00:01:12.600000  5CB1E4          (45.508525, -73.591063)   

   distance_from_home  friend_top_time   id   latitude  longitude  \
4            1.580895         00:01:54  293  45.510657 -73.593420   
1            1.559217         00:01:07  290  45.513068 -73.589766   
7            1.909704         00:00:53  386  45.517084 -73.564973   
3            1.990143  

In [7]:
final_df

,average_total_top_time,color,coordinates,distance_from_home,friend_top_time,id,latitude,longitude,name,segment_distance,...,segment_name,total_top_time,type,url,friend_versus_top1,friend_versus_top10,distance_from_home_Ranked,segment_distance_Ranked,friend_versus_top10_Ranked,target_score
0,00:00:22.100000,FFEE33,"(45.502192, -73.577163)",1.599334,00:00:58,1537,45.502192,-73.577163,Peel Push - Sherbrooke to Docteur Penfield : ...,204.300,...,Peel Push - Sherbrooke to Docteur Penfield,00:00:19,Sasha Wloski,http://www.strava.com/segments/7849363/compare/,205.263158,162.443439,59,10,1,13.5
1,00:00:28.666666,BA9778,"(45.5021951906, -73.5629598051)",1.936012,00:00:39,1538,45.502195,-73.562960,Beaver Hall: [Josh Felker],237.550,...,Beaver Hall,00:00:20,Josh Felker,http://www.strava.com/segments/1176830/compare/,95.000000,36.046515,86,16,26,37.0
2,00:00:51.400000,FFEE33,"(45.4916093499, -73.5882798583)",0.984138,00:01:25,1539,45.491609,-73.588280,Atwater/Saint Sulpice Short Safe: [Sasha Wloski],411.851,...,Atwater/Saint Sulpice Short Safe,00:00:40,Sasha Wloski,http://www.strava.com/segments/1840013/compare/,112.500000,65.369650,23,33,12,16.3
3,00:00:58.400000,5CB1E4,"(45.49695, -73.586141)",1.246526,00:01:15,1540,45.496950,-73.586141,DrPenfield: [Eoin Craigie],836.300,...,DrPenfield,00:00:53,Eoin Craigie,http://www.strava.com/segments/7381906/compare/,41.509434,28.424658,36,63,31,35.2
4,00:00:22.200000,FFEE33,"(45.4852914065, -73.5987708997)",1.731428,00:00:26,854,45.485291,-73.598771,Av Church Hill: [Sasha Wloski],176.390,...,Av Church Hill,00:00:19,Sasha Wloski,http://www.strava.com/segments/2477484/compare/,36.842105,17.117117,62,6,46,45.2
5,00:01:08.200000,FFEE33,"(45.4995532965, -73.5760966161)",1.307503,00:01:22,1554,45.499553,-73.576097,Maisonneuve - Drummond/University: [Sasha Wlo...,683.419,...,Maisonneuve - Drummond/University,00:01:01,Sasha Wloski,http://www.strava.com/segments/1919327/compare/,34.426230,20.234604,40,55,42,42.9
6,00:00:14.500000,A03D3D,"(45.505209, -73.570691)",1.994722,00:00:30,1580,45.505209,-73.570691,Montée Aylmer (courte): [Laurie McQueen],207.300,...,Montée Aylmer (courte),00:00:11,Laurie McQueen,http://www.strava.com/segments/8435226/compare/,172.727273,106.896552,94,12,6,24.2
7,00:00:35.900000,5CB1E4,"(45.5041246293, -73.5785079039)",1.818288,00:01:04,1589,45.504125,-73.578508,McGill-a Gorilla: [Eoin Craigie],454.051,...,McGill-a Gorilla,00:00:34,Eoin Craigie,http://www.strava.com/segments/2227344/compare/,88.235294,78.272981,77,38,9,25.5
8,00:02:09,A03D3D,"(45.503388, -73.566659)",1.908587,00:02:59,1603,45.503388,-73.566659,Rue Alymer N. (René-Lévesque / Prince-Arthur):...,1081.000,...,Rue Alymer N. (René-Lévesque / Prince-Arthur),00:01:50,Laurie McQueen,http://www.strava.com/segments/8435223/compare/,62.727273,38.759690,84,71,23,40.0
9,00:00:40.733333,FF9233,"(45.472259, -73.580563)",1.750990,00:00:38,194,45.472259,-73.580563,Short sprint to underpass: [Mario Born],453.600,...,Short sprint to underpass,00:00:27,Mario Born,http://www.strava.com/segments/7824610/compare/,40.740741,-6.710310,66,37,75,69.4
